In [1]:
import pandas as pd
import numpy as np

In [2]:
import pickle
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [3]:
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input
from keras.layers import Embedding, LeakyReLU,BatchNormalization
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalMaxPool1D
from keras.layers import LSTM, Bidirectional
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

In [4]:
from laserembeddings import Laser

laser = Laser()

In [5]:
dt = pd.read_csv('train.csv')
dt

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


In [76]:
x_p = dt.premise.values
x_h = dt.hypothesis.values
y = dt.label.values
z = dt.lang_abv.values
# split train and test
x_train_p, x_test_p,x_train_h,x_test_h, y_train, y_test,z_train,z_test = train_test_split(x_p,x_h, y,z, test_size=0.3, random_state=0)

In [7]:
x_train_p

array(['เมื่อการตัดสินใจเหล่านี้ได้เกิดขึ้นไปแล้ว, องค์กร CIO ต้องให้การสนับสนุนที่มีประสิทธิภาพและพร้อมตอบสนองผ่านการจัดสรรทรัพยากรที่มีประสิทธิภาพและการดำเนินการตามหน้าที่ในแต่ละวัน',
       "Do you know how long we've been here? he asked one morning as they sat facing each other at breakfast.",
       "Look for these items in the picturesque open-air market of Sa Penya (Ibiza Town) or for a wider selection at the bustling, covered central market in the newer part of town (carrer d'Extremadura).",
       ...,
       'Anfang Mai 1996 bekam die CIA mit, dass Bin Laden den Sudan vielleicht verlassen hatte.',
       'Sie war es nun, die sich verteidigte, ihre Stimme zitterte vor Entrüstung.',
       'उन्होंने कहा, हम आपके लिए रहने के लिए एक जगह का भुगतान कर रहे हैं।'],
      dtype=object)

In [8]:
emb_p = laser.embed_sentences([x for x in x_train_p], lang = z_train)
emb_h = laser.embed_sentences([x for x in x_train_h], lang = z_train)

In [9]:
emb_p.shape

(8484, 1024)

In [46]:
embeddings = np.hstack((emb_p,emb_h))

In [47]:
embeddings.shape

(8484, 2048)

In [12]:
test_p = laser.embed_sentences([x for x in x_test_p], lang = z_test)
test_h = laser.embed_sentences([x for x in x_test_h], lang = z_test)

In [13]:
test_p.shape

(3636, 1024)

In [14]:
test_emb = np.hstack((test_p,test_h))

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
model = RandomForestClassifier()
model.fit(embeddings,y_train)

predictions = model.predict(test_emb)
print('Classification report:')
print(classification_report(y_test, predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test, predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.45      0.55      0.49      1237
           1       0.42      0.37      0.40      1147
           2       0.60      0.53      0.56      1252

    accuracy                           0.49      3636
   macro avg       0.49      0.48      0.48      3636
weighted avg       0.49      0.49      0.48      3636

Confusion matrix:
[[677 331 229]
 [509 429 209]
 [331 263 658]]


In [18]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [17]:
# data preprocessing
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

scaler =  MinMaxScaler()
scaler.fit(embeddings)
X_train = scaler.transform(embeddings)
X_test = scaler.transform(test_emb)

In [19]:
param_grid = [{'sel__k': [ 10, all],
    'learner__max_depth': [None, 100],
    #'learner__max_features': [2, 3],
    'learner__min_samples_leaf': [1, 4],
    #'learner__min_samples_split': [8, 10, 12],
    'learner__n_estimators': [100,300]}]

pipe = Pipeline([
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', RandomForestClassifier(bootstrap = True))  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
pipe_search = GridSearchCV(pipe, param_grid, cv=5, n_jobs = n_jobs, verbose=True).fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\eleon\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\eleon\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\eleon\anaconda3\lib\site-packages\sklearn\pipeline.py", line 378, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\eleon\anaconda3\lib\site-packages\sklearn\pipeline.py", line 336, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "C:\Users\eleon\anacond

In [20]:
predictions = pipe_search.best_estimator_.predict(X_test)
print('Classification report:')
print(classification_report(y_test, predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test, predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.44      0.56      0.49      1237
           1       0.39      0.31      0.35      1147
           2       0.61      0.55      0.58      1252

    accuracy                           0.48      3636
   macro avg       0.48      0.47      0.47      3636
weighted avg       0.48      0.48      0.47      3636

Confusion matrix:
[[694 324 219]
 [562 358 227]
 [330 236 686]]


In [21]:
# without pipeline
clf = LinearSVC(C=1.0, random_state=42)
clf.fit(embeddings, y_train)

y_pred = clf.predict(test_emb)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.49807480748074806
F1-score [0.48009368 0.44395988 0.56847331]
              precision    recall  f1-score   support

           0       0.46      0.50      0.48      1237
           1       0.44      0.44      0.44      1147
           2       0.59      0.55      0.57      1252

    accuracy                           0.50      3636
   macro avg       0.50      0.50      0.50      3636
weighted avg       0.50      0.50      0.50      3636



In [22]:
#with pipeline and grid serach
param_grid = [{'sel__k': [8, 10, 50,'all'], 'learner__C': [0.1, 0.5, 1, 10]}]

opt_pipeline = Pipeline([
    
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', LinearSVC())  # learning algorithm
])

n_jobs = 3 #Number of jobs to run in parallel
SVM_opt_search = GridSearchCV(opt_pipeline, param_grid, n_jobs = n_jobs, verbose=True).fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [23]:
opt_predictions = SVM_opt_search.best_estimator_.predict(X_test)

print('Classification report:')
print(classification_report(y_test, opt_predictions, zero_division = 1))
print('Confusion matrix:')
cm = confusion_matrix(y_test, opt_predictions)
print(cm)

Classification report:
              precision    recall  f1-score   support

           0       0.47      0.51      0.49      1237
           1       0.44      0.42      0.43      1147
           2       0.61      0.58      0.60      1252

    accuracy                           0.51      3636
   macro avg       0.51      0.51      0.51      3636
weighted avg       0.51      0.51      0.51      3636

Confusion matrix:
[[628 373 236]
 [437 487 223]
 [267 254 731]]


## neural network

In [52]:
# Convert y to OHE 
y_train = to_categorical(y_train, 3)
y_test  = to_categorical(y_test, 3)

In [59]:
input = Input(shape=(embeddings.shape[1], )) 
x   =  Dense(64, activation="relu")(input)
x   =  Dropout(0.2)(x)
x   =  Dense(3, activation="softmax")(x)
model = Model(inputs=input, outputs=x)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [60]:
model.fit(embeddings,y_train,epochs=10,validation_split=0.2)

Epoch 1/10
213/213 [==============================] - 1s 2ms/step - loss: 1.0849 - accuracy: 0.3877 - val_loss: 1.0602 - val_accuracy: 0.4697
Epoch 2/10
213/213 [==============================] - 0s 1ms/step - loss: 1.0331 - accuracy: 0.4880 - val_loss: 1.0329 - val_accuracy: 0.4903
Epoch 3/10
213/213 [==============================] - 0s 2ms/step - loss: 0.9846 - accuracy: 0.5250 - val_loss: 0.9999 - val_accuracy: 0.5209
Epoch 4/10
213/213 [==============================] - 0s 1ms/step - loss: 0.9552 - accuracy: 0.5468 - val_loss: 0.9852 - val_accuracy: 0.5303
Epoch 5/10
213/213 [==============================] - 0s 1ms/step - loss: 0.9232 - accuracy: 0.5692 - val_loss: 0.9788 - val_accuracy: 0.5345
Epoch 6/10
213/213 [==============================] - 0s 1ms/step - loss: 0.8989 - accuracy: 0.5804 - val_loss: 0.9757 - val_accuracy: 0.5262
Epoch 7/10
213/213 [==============================] - 0s 1ms/step - loss: 0.8735 - accuracy: 0.6035 - val_loss: 0.9930 - val_accuracy: 0.5156
Epoch 

In [61]:
c = np.round(model.predict(test_emb))

114/114 [==============================] - 0s 716us/step


In [62]:
print('Accuracy %s' % accuracy_score(y_test, c))
print('F1-score %s' % f1_score(y_test, c,average=None,zero_division = 0))
print('Classification report:')
print(classification_report(y_test, c))

Accuracy 0.3844884488448845
F1-score [0.41120608 0.41734694 0.54111922]
Classification report:
              precision    recall  f1-score   support

           0       0.50      0.35      0.41      1237
           1       0.50      0.36      0.42      1147
           2       0.69      0.44      0.54      1252

   micro avg       0.56      0.38      0.46      3636
   macro avg       0.56      0.38      0.46      3636
weighted avg       0.57      0.38      0.46      3636
 samples avg       0.38      0.38      0.38      3636



C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
input = Input(shape=(embeddings.shape[1], )) 
x = Dense(512, activation='relu')(input)

x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)

x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)

x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)

x = Dropout(0.2)(x)
x = Dense(8, activation='relu')(x)

x = Dropout(0.2)(x)
x = Dense(3, activation="softmax")(x)
mod = Model(inputs=input, outputs=x)
mod.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [70]:
mod.fit(embeddings,y_train,epochs=1000,validation_split=0.2)

Epoch 1/1000
213/213 [==============================] - 2s 7ms/step - loss: 1.0965 - accuracy: 0.3554 - val_loss: 1.0842 - val_accuracy: 0.4313
Epoch 2/1000
213/213 [==============================] - 1s 7ms/step - loss: 1.0546 - accuracy: 0.4251 - val_loss: 1.0554 - val_accuracy: 0.3907
Epoch 3/1000
213/213 [==============================] - 1s 7ms/step - loss: 1.0223 - accuracy: 0.4559 - val_loss: 1.0140 - val_accuracy: 0.4596
Epoch 4/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.9942 - accuracy: 0.4811 - val_loss: 0.9970 - val_accuracy: 0.4803
Epoch 5/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.9718 - accuracy: 0.4961 - val_loss: 1.0023 - val_accuracy: 0.4897
Epoch 6/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.9580 - accuracy: 0.5002 - val_loss: 0.9956 - val_accuracy: 0.4879
Epoch 7/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.9290 - accuracy: 0.5167 - val_loss: 1.0049 - val_accuracy:

213/213 [==============================] - 1s 7ms/step - loss: 0.2163 - accuracy: 0.9257 - val_loss: 3.8341 - val_accuracy: 0.4838
Epoch 58/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.1977 - accuracy: 0.9268 - val_loss: 4.2343 - val_accuracy: 0.4814
Epoch 59/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.1768 - accuracy: 0.9414 - val_loss: 3.9006 - val_accuracy: 0.4755
Epoch 60/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.1827 - accuracy: 0.9411 - val_loss: 3.7728 - val_accuracy: 0.4667
Epoch 61/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.1712 - accuracy: 0.9406 - val_loss: 3.9990 - val_accuracy: 0.4814
Epoch 62/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.1797 - accuracy: 0.9381 - val_loss: 3.3373 - val_accuracy: 0.4826
Epoch 63/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.1629 - accuracy: 0.9453 - val_loss: 3.9000 - val_accuracy: 0.4862

213/213 [==============================] - 1s 7ms/step - loss: 0.0876 - accuracy: 0.9713 - val_loss: 5.4351 - val_accuracy: 0.4720
Epoch 114/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.1125 - accuracy: 0.9661 - val_loss: 4.2867 - val_accuracy: 0.4814
Epoch 115/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0731 - accuracy: 0.9748 - val_loss: 6.3569 - val_accuracy: 0.4897
Epoch 116/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.1337 - accuracy: 0.9601 - val_loss: 5.0721 - val_accuracy: 0.4655
Epoch 117/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.1236 - accuracy: 0.9598 - val_loss: 4.9090 - val_accuracy: 0.4879
Epoch 118/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0917 - accuracy: 0.9724 - val_loss: 4.4767 - val_accuracy: 0.4873
Epoch 119/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0891 - accuracy: 0.9710 - val_loss: 5.1470 - val_accuracy: 

213/213 [==============================] - 1s 6ms/step - loss: 0.0671 - accuracy: 0.9783 - val_loss: 6.1841 - val_accuracy: 0.4826
Epoch 170/1000
213/213 [==============================] - 1s 6ms/step - loss: 0.0750 - accuracy: 0.9782 - val_loss: 6.1761 - val_accuracy: 0.4720
Epoch 171/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0970 - accuracy: 0.9688 - val_loss: 4.9981 - val_accuracy: 0.4808
Epoch 172/1000
213/213 [==============================] - 1s 6ms/step - loss: 0.0702 - accuracy: 0.9766 - val_loss: 5.7899 - val_accuracy: 0.4867
Epoch 173/1000
213/213 [==============================] - 1s 6ms/step - loss: 0.0817 - accuracy: 0.9775 - val_loss: 5.6789 - val_accuracy: 0.4820
Epoch 174/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0808 - accuracy: 0.9747 - val_loss: 5.0667 - val_accuracy: 0.4903
Epoch 175/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0744 - accuracy: 0.9755 - val_loss: 5.5151 - val_accuracy: 

213/213 [==============================] - 1s 7ms/step - loss: 0.0886 - accuracy: 0.9723 - val_loss: 6.1742 - val_accuracy: 0.4909
Epoch 226/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0717 - accuracy: 0.9766 - val_loss: 7.0018 - val_accuracy: 0.4909
Epoch 227/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0566 - accuracy: 0.9817 - val_loss: 6.9845 - val_accuracy: 0.4968
Epoch 228/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0673 - accuracy: 0.9779 - val_loss: 6.8926 - val_accuracy: 0.4867
Epoch 229/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0609 - accuracy: 0.9820 - val_loss: 6.7727 - val_accuracy: 0.4962
Epoch 230/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.0580 - accuracy: 0.9823 - val_loss: 6.9558 - val_accuracy: 0.5044
Epoch 231/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0854 - accuracy: 0.9726 - val_loss: 7.0323 - val_accuracy: 

...

Epoch 729/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0485 - accuracy: 0.9913 - val_loss: 14.8808 - val_accuracy: 0.4808
Epoch 730/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0433 - accuracy: 0.9854 - val_loss: 16.6153 - val_accuracy: 0.4732
Epoch 731/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0561 - accuracy: 0.9854 - val_loss: 13.7834 - val_accuracy: 0.4885
Epoch 732/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0418 - accuracy: 0.9903 - val_loss: 13.7804 - val_accuracy: 0.4856
Epoch 733/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0796 - accuracy: 0.9831 - val_loss: 10.0412 - val_accuracy: 0.4785
Epoch 734/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0277 - accuracy: 0.9917 - val_loss: 12.0128 - val_accuracy: 0.4820
Epoch 735/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0237 - accuracy: 0.9932 - val_loss: 11

Epoch 785/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0188 - accuracy: 0.9940 - val_loss: 18.0169 - val_accuracy: 0.4944
Epoch 786/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0170 - accuracy: 0.9954 - val_loss: 18.8150 - val_accuracy: 0.4856
Epoch 787/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0228 - accuracy: 0.9944 - val_loss: 17.1411 - val_accuracy: 0.4838
Epoch 788/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0449 - accuracy: 0.9881 - val_loss: 12.0414 - val_accuracy: 0.4844
Epoch 789/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0249 - accuracy: 0.9928 - val_loss: 15.4187 - val_accuracy: 0.4891
Epoch 790/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0284 - accuracy: 0.9909 - val_loss: 14.1852 - val_accuracy: 0.4856
Epoch 791/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0230 - accuracy: 0.9937 - val_loss: 15

Epoch 841/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0276 - accuracy: 0.9940 - val_loss: 18.4905 - val_accuracy: 0.4873
Epoch 842/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.0334 - accuracy: 0.9923 - val_loss: 16.4850 - val_accuracy: 0.4791
Epoch 843/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0414 - accuracy: 0.9909 - val_loss: 16.9360 - val_accuracy: 0.4732
Epoch 844/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0391 - accuracy: 0.9888 - val_loss: 15.1971 - val_accuracy: 0.4791
Epoch 845/1000
213/213 [==============================] - 2s 8ms/step - loss: 0.0389 - accuracy: 0.9888 - val_loss: 15.2019 - val_accuracy: 0.4838
Epoch 846/1000
213/213 [==============================] - 2s 8ms/step - loss: 0.0191 - accuracy: 0.9940 - val_loss: 15.3583 - val_accuracy: 0.4926
Epoch 847/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0196 - accuracy: 0.9938 - val_loss: 18

Epoch 897/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.0160 - accuracy: 0.9948 - val_loss: 15.8419 - val_accuracy: 0.4968
Epoch 898/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.0238 - accuracy: 0.9940 - val_loss: 13.2931 - val_accuracy: 0.4803
Epoch 899/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.0132 - accuracy: 0.9956 - val_loss: 18.5835 - val_accuracy: 0.4826
Epoch 900/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.0239 - accuracy: 0.9934 - val_loss: 17.1763 - val_accuracy: 0.4803
Epoch 901/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.0244 - accuracy: 0.9943 - val_loss: 15.7567 - val_accuracy: 0.4808
Epoch 902/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.0253 - accuracy: 0.9932 - val_loss: 19.5325 - val_accuracy: 0.4791
Epoch 903/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.0265 - accuracy: 0.9903 - val_loss: 16

Epoch 953/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0237 - accuracy: 0.9941 - val_loss: 17.7794 - val_accuracy: 0.4808
Epoch 954/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0321 - accuracy: 0.9922 - val_loss: 13.0119 - val_accuracy: 0.4791
Epoch 955/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0291 - accuracy: 0.9931 - val_loss: 14.2759 - val_accuracy: 0.4826
Epoch 956/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0274 - accuracy: 0.9938 - val_loss: 14.7687 - val_accuracy: 0.4926
Epoch 957/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0377 - accuracy: 0.9909 - val_loss: 12.3539 - val_accuracy: 0.4985
Epoch 958/1000
213/213 [==============================] - 1s 7ms/step - loss: 0.0274 - accuracy: 0.9910 - val_loss: 13.4450 - val_accuracy: 0.4956
Epoch 959/1000
213/213 [==============================] - 2s 7ms/step - loss: 0.0277 - accuracy: 0.9919 - val_loss: 14

In [71]:
pred = np.round(mod.predict(test_emb))

114/114 [==============================] - 0s 2ms/step


In [72]:
print('Accuracy %s' % accuracy_score(y_test, pred))
print('F1-score %s' % f1_score(y_test, pred,average=None,zero_division = 0))
print('Classification report:')
print(classification_report(y_test, pred))

Accuracy 0.4829482948294829
F1-score [0.46669234 0.433213   0.55121349]
Classification report:
              precision    recall  f1-score   support

           0       0.45      0.49      0.47      1237
           1       0.45      0.42      0.43      1147
           2       0.57      0.54      0.55      1252

   micro avg       0.49      0.48      0.48      3636
   macro avg       0.49      0.48      0.48      3636
weighted avg       0.49      0.48      0.49      3636
 samples avg       0.48      0.48      0.48      3636



C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
def cos():
    c = []
    for i in range(len(emb_p)):
        #print(i)
        c.append(cosine_similarity(emb_p[i].reshape(1,-1),emb_h[i].reshape(1,-1)))
    return c

In [27]:
cosine = np.array(cos())

In [28]:
cosine.shape

(8484, 1, 1)

In [29]:
cosine = np.squeeze(cosine)
cosine = np.expand_dims(cosine, axis=1)
cosine.shape

(8484, 1)

In [30]:
emb_2 = np.hstack((emb_p,emb_h,cosine))

In [31]:
def cos2():
    l = []
    for i in range(len(test_p)):
        #print(i)
        l.append(cosine_similarity(test_p[i].reshape(1,-1),test_h[i].reshape(1,-1)))
    return l

In [32]:
cosine_t = np.array(cos2())

In [33]:
cosine_t = np.squeeze(cosine_t)
cosine_t = np.expand_dims(cosine_t, axis=1)
cosine_t.shape

(3636, 1)

In [37]:
test2 = np.hstack((test_p,test_h,cosine_t))

In [73]:
input = Input(shape=(emb_2.shape[1], )) 
x = Dense(512, activation=LeakyReLU())(input)

x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)

x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)

x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)

x = Dropout(0.2)(x)
x = Dense(8, activation='relu')(x)

x = Dropout(0.2)(x)
x = Dense(3, activation="softmax")(x)
mod1 = Model(inputs=input, outputs=x)
mod1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
mod1.fit(emb_2,y_train,epochs=200,validation_split=0.2)

Epoch 1/200
213/213 [==============================] - 2s 8ms/step - loss: 1.0987 - accuracy: 0.3435 - val_loss: 1.0923 - val_accuracy: 0.3730
Epoch 2/200
213/213 [==============================] - 2s 7ms/step - loss: 1.0870 - accuracy: 0.3878 - val_loss: 1.0689 - val_accuracy: 0.4072
Epoch 3/200
213/213 [==============================] - 2s 7ms/step - loss: 1.0491 - accuracy: 0.4498 - val_loss: 1.0318 - val_accuracy: 0.4726
Epoch 4/200
213/213 [==============================] - 2s 7ms/step - loss: 1.0130 - accuracy: 0.4942 - val_loss: 0.9924 - val_accuracy: 0.5080
Epoch 5/200
213/213 [==============================] - 2s 8ms/step - loss: 0.9768 - accuracy: 0.5269 - val_loss: 0.9739 - val_accuracy: 0.5351
Epoch 6/200
213/213 [==============================] - 1s 7ms/step - loss: 0.9296 - accuracy: 0.5623 - val_loss: 0.9736 - val_accuracy: 0.5351
Epoch 7/200
213/213 [==============================] - 1s 7ms/step - loss: 0.9013 - accuracy: 0.5877 - val_loss: 0.9661 - val_accuracy: 0.5374

Epoch 58/200
213/213 [==============================] - 1s 7ms/step - loss: 0.1446 - accuracy: 0.9509 - val_loss: 3.9588 - val_accuracy: 0.5268
Epoch 59/200
213/213 [==============================] - 1s 7ms/step - loss: 0.1315 - accuracy: 0.9549 - val_loss: 3.7943 - val_accuracy: 0.5056
Epoch 60/200
213/213 [==============================] - 1s 7ms/step - loss: 0.1425 - accuracy: 0.9511 - val_loss: 3.8226 - val_accuracy: 0.5268
Epoch 61/200
213/213 [==============================] - 1s 7ms/step - loss: 0.1299 - accuracy: 0.9555 - val_loss: 3.5983 - val_accuracy: 0.5221
Epoch 62/200
213/213 [==============================] - 1s 6ms/step - loss: 0.1279 - accuracy: 0.9579 - val_loss: 4.5907 - val_accuracy: 0.5150
Epoch 63/200
213/213 [==============================] - 1s 7ms/step - loss: 0.1472 - accuracy: 0.9490 - val_loss: 3.9329 - val_accuracy: 0.5292
Epoch 64/200
213/213 [==============================] - 1s 7ms/step - loss: 0.1305 - accuracy: 0.9562 - val_loss: 4.1105 - val_accuracy:

Epoch 115/200
213/213 [==============================] - 1s 7ms/step - loss: 0.0655 - accuracy: 0.9813 - val_loss: 4.8870 - val_accuracy: 0.5186
Epoch 116/200
213/213 [==============================] - 1s 7ms/step - loss: 0.0531 - accuracy: 0.9820 - val_loss: 5.3262 - val_accuracy: 0.5245
Epoch 117/200
213/213 [==============================] - 1s 7ms/step - loss: 0.0731 - accuracy: 0.9747 - val_loss: 4.8647 - val_accuracy: 0.5056
Epoch 118/200
213/213 [==============================] - 1s 7ms/step - loss: 0.0578 - accuracy: 0.9795 - val_loss: 5.7782 - val_accuracy: 0.5156
Epoch 119/200
213/213 [==============================] - 1s 7ms/step - loss: 0.1043 - accuracy: 0.9695 - val_loss: 5.2537 - val_accuracy: 0.5097
Epoch 120/200
213/213 [==============================] - 1s 7ms/step - loss: 0.1153 - accuracy: 0.9635 - val_loss: 4.1806 - val_accuracy: 0.5144
Epoch 121/200
213/213 [==============================] - 1s 7ms/step - loss: 0.0607 - accuracy: 0.9814 - val_loss: 5.1525 - val_ac

213/213 [==============================] - 2s 7ms/step - loss: 0.0466 - accuracy: 0.9850 - val_loss: 5.5237 - val_accuracy: 0.5032
Epoch 172/200
213/213 [==============================] - 2s 7ms/step - loss: 0.0475 - accuracy: 0.9863 - val_loss: 6.1353 - val_accuracy: 0.5221
Epoch 173/200
213/213 [==============================] - 2s 7ms/step - loss: 0.0701 - accuracy: 0.9803 - val_loss: 5.1627 - val_accuracy: 0.5309
Epoch 174/200
213/213 [==============================] - 2s 7ms/step - loss: 0.0589 - accuracy: 0.9823 - val_loss: 5.6604 - val_accuracy: 0.5268
Epoch 175/200
213/213 [==============================] - 2s 7ms/step - loss: 0.0714 - accuracy: 0.9804 - val_loss: 4.6077 - val_accuracy: 0.5192
Epoch 176/200
213/213 [==============================] - 2s 8ms/step - loss: 0.0446 - accuracy: 0.9864 - val_loss: 5.4444 - val_accuracy: 0.5209
Epoch 177/200
213/213 [==============================] - 2s 8ms/step - loss: 0.0429 - accuracy: 0.9842 - val_loss: 6.3151 - val_accuracy: 0.5256

In [74]:
pred1 = np.round(mod1.predict(test2))

114/114 [==============================] - 0s 2ms/step


In [75]:
print('Accuracy %s' % accuracy_score(y_test, pred1))
print('F1-score %s' % f1_score(y_test, pred1,average=None,zero_division = 0))
print('Classification report:')
print(classification_report(y_test, pred1))

Accuracy 0.4991749174917492
F1-score [0.51494619 0.45753425 0.53015873]
Classification report:
              precision    recall  f1-score   support

           0       0.51      0.52      0.51      1237
           1       0.48      0.44      0.46      1147
           2       0.53      0.53      0.53      1252

   micro avg       0.51      0.50      0.50      3636
   macro avg       0.51      0.50      0.50      3636
weighted avg       0.51      0.50      0.50      3636
 samples avg       0.50      0.50      0.50      3636



C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
input = Input(shape=(emb_2.shape[1], )) 
#x   =  LSTM(100, return_sequences=True,name='lstm_layer')(input)
#x   =  GlobalMaxPool1D()(input)
x   =  Dense(64, activation="relu")(input)
x   =  Dropout(0.2)(x)
x   =  Dense(3, activation="softmax")(x)
model3 = Model(inputs=input, outputs=x)
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model3.fit(emb_2,y_train,epochs=10,validation_split=0.2)

Epoch 1/10
213/213 [==============================] - 1s 2ms/step - loss: 1.0834 - accuracy: 0.4000 - val_loss: 1.0604 - val_accuracy: 0.4973
Epoch 2/10
213/213 [==============================] - 0s 1ms/step - loss: 1.0275 - accuracy: 0.5024 - val_loss: 1.0332 - val_accuracy: 0.4714
Epoch 3/10
213/213 [==============================] - 0s 1ms/step - loss: 0.9759 - accuracy: 0.5450 - val_loss: 0.9784 - val_accuracy: 0.5504
Epoch 4/10
213/213 [==============================] - 0s 1ms/step - loss: 0.9335 - accuracy: 0.5746 - val_loss: 0.9607 - val_accuracy: 0.5545
Epoch 5/10
213/213 [==============================] - 0s 1ms/step - loss: 0.8995 - accuracy: 0.5948 - val_loss: 0.9609 - val_accuracy: 0.5410
Epoch 6/10
213/213 [==============================] - 0s 1ms/step - loss: 0.8685 - accuracy: 0.6085 - val_loss: 0.9445 - val_accuracy: 0.5598
Epoch 7/10
213/213 [==============================] - 0s 1ms/step - loss: 0.8403 - accuracy: 0.6287 - val_loss: 0.9373 - val_accuracy: 0.5645
Epoch 

In [67]:
pred3 = np.round(model3.predict(test2))

114/114 [==============================] - 0s 688us/step


In [68]:
print('Accuracy %s' % accuracy_score(y_test, pred3))
print('F1-score %s' % f1_score(y_test, pred3,average=None,zero_division = 0))
print('Classification report:')
print(classification_report(y_test, pred3))

Accuracy 0.4152915291529153
F1-score [0.39978034 0.482494   0.57385096]
Classification report:
              precision    recall  f1-score   support

           0       0.62      0.29      0.40      1237
           1       0.54      0.44      0.48      1147
           2       0.65      0.51      0.57      1252

   micro avg       0.60      0.42      0.49      3636
   macro avg       0.60      0.42      0.49      3636
weighted avg       0.61      0.42      0.49      3636
 samples avg       0.42      0.42      0.42      3636



C:\Users\eleon\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
# without pipeline
clf = LinearSVC(C=1.0, random_state=42)
clf.fit(emb_2, y_train)

y_pred = clf.predict(test2)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.5657315731573157
F1-score [0.58716981 0.54075372 0.56581197]
              precision    recall  f1-score   support

           0       0.55      0.63      0.59      1237
           1       0.54      0.54      0.54      1147
           2       0.61      0.53      0.57      1252

    accuracy                           0.57      3636
   macro avg       0.57      0.57      0.56      3636
weighted avg       0.57      0.57      0.57      3636

